In [30]:
import pandas as pd
import datetime as dt

In [31]:
from collections import Counter
import math
from scipy import stats
import numpy as np
import tqdm

In [32]:
train_df = pd.read_csv('train.csv')

In [33]:
test_df = pd.read_csv('test_bqCt9Pv.csv')

In [34]:
train_df.columns

Index(['UniqueID', 'disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID', 'Date.of.Birth',
       'Employment.Type', 'DisbursalDate', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PERFORM_CNS.SCORE.DESCRIPTION', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS',
       'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH', 'NO.OF_INQUIRIES',
       'loan_default'],
      dtype='object')

### Feature correlation

In [35]:
# import matplotlib.pyplot as plt
# # plt.style.use('ggplot') 
# import seaborn as sns
# plt.figure(figsize=(20,20))
# sns.heatmap(train_df.corr(), annot=True, fmt=".2f")

# plt.show()

In [37]:
def remove_outlier(df, bad_count=3000):

    temp = df.drop(['Employment.Type', "PERFORM_CNS.SCORE.DESCRIPTION"], axis=1)
    temp = temp.dropna(axis=1)
    bad_list = []
    for column in tqdm.tqdm(temp.columns):
        try:
            bads = temp[np.abs(temp[column]-temp[column].mean()) > (4*temp[column].std())]
            bad_list = bad_list + bads.UniqueID.tolist()
#             print(column)
        except:
            pass
    #         print(column , " issue")
    bad_count = max(bad_count, len(bad_list)-1)
    bads = dict(Counter(bad_list).most_common(bad_count))
    badids = list(bads.keys())
    bad_mask = df.UniqueID.isin(badids)
    df = df[~bad_mask]
    return df

In [38]:
def is_more(a,b):
    if a>b:
        return 1
    return 0

In [39]:
def con(xx):
    #  convert date month to int
    xx = xx.split()
    yr = xx[0].replace('yrs','')
    
    mn = xx[1].replace('mon','')
    return int(yr)*365+int(mn)*30

In [40]:
def preprocess(df):
    df['Secondary_amount'] = df['SEC.SANCTIONED.AMOUNT'] + df['SEC.DISBURSED.AMOUNT']
    df = df.drop(['SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT'], axis=1)
    df['sec_greater']=df.apply(lambda x: is_more(x['SEC.INSTAL.AMT'], x['PRIMARY.INSTAL.AMT']), axis=1)
    
    df['Date.of.Birth'] = pd.to_datetime(df['Date.of.Birth'])
    df['DisbursalDate'] = pd.to_datetime(df['DisbursalDate'])


    NOW=dt.datetime(2019,3,1)
    
    df['Date.of.Birth'] = abs(df['Date.of.Birth']-NOW).dt.days//365
    df['DisbursalDate'] = abs(df['DisbursalDate']-NOW)
    df['DisbursalDate'] = df['DisbursalDate'].dt.days 
    
    df['AVERAGE.ACCT.AGE'] = df['AVERAGE.ACCT.AGE'].apply(lambda x: con(x))
    df['CREDIT.HISTORY.LENGTH'] = df['CREDIT.HISTORY.LENGTH'].apply(lambda x: con(x))
    
#     df['Employment.Type'] = df["Employment.Type"].fillna("none", inplace=True)    
    
    #remove collinear features
    #df['combined_flag'] = df['MobileNo_Avl_Flag'] or df['Aadhar_flag']+df['PAN_flag']+df['VoterID_flag']+df['Driving_flag']+df['Passport_flag']
    df = df.drop(['MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag', 'Driving_flag', 'Passport_flag'], axis=1)
    df = df.drop(['SEC.CURRENT.BALANCE', 'Secondary_amount', 'SEC.INSTAL.AMT' ,'SEC.OVERDUE.ACCTS','SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS', 'sec_greater'], axis=1)
    
    ## custom features
    df['cust_disbursal_ratio'] = df['PRI.SANCTIONED.AMOUNT'] / df['disbursed_amount']
    df["cust_log_disbursed_amount"] = df["disbursed_amount"].apply(math.log)
    df["cust_log_asset_cost"] = df["asset_cost"].apply(math.log)
    df["cust_acc_ratio"] = df['PRI.ACTIVE.ACCTS'] / df["PRI.NO.OF.ACCTS"] 
    df["cust_acc_ratio"] =  df["cust_acc_ratio"].fillna(0, inplace=True)
    
    df = df.drop(['disbursed_amount','PRI.SANCTIONED.AMOUNT', 'PRI.NO.OF.ACCTS', 'AVERAGE.ACCT.AGE', 'asset_cost'], axis=1)
    
    return df
    

In [41]:
train_df = remove_outlier(train_df,bad_count= 5000)

100%|████████████████████████████████████████████████████████████████████████████████| 39/39 [08:51<00:00, 13.63s/it]


In [42]:
train_df = preprocess(train_df)
test_df = preprocess(test_df)

In [43]:
train_df["Employment.Type"] = train_df["Employment.Type"].fillna("no")
test_df["Employment.Type"] = test_df["Employment.Type"].fillna("no")

## Split 

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(train_df.drop(['UniqueID','loan_default'], axis=1), train_df["loan_default"], train_size=0.85, random_state=1244)

C:\Users\SAM\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Categorical features

In [45]:
cat_features = ['branch_id', 'supplier_id', 'manufacturer_id', 'State_ID', 'Employee_code_ID', 'Current_pincode_ID','Employment.Type','PERFORM_CNS.SCORE.DESCRIPTION']

## Catboost classifier

In [46]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    loss_function='Logloss',
    custom_loss=['AUC'],
    use_best_model=True,
    task_type='GPU',
    early_stopping_rounds=50,
#     class_weights = [1, 2.3]
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    verbose=True
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

0:	learn: 0.6732112	test: 0.6726414	best: 0.6726414 (0)	total: 82.8ms	remaining: 1m 22s
1:	learn: 0.6551901	test: 0.6544989	best: 0.6544989 (1)	total: 154ms	remaining: 1m 16s
2:	learn: 0.6393858	test: 0.6386216	best: 0.6386216 (2)	total: 235ms	remaining: 1m 18s
3:	learn: 0.6251298	test: 0.6242195	best: 0.6242195 (3)	total: 310ms	remaining: 1m 17s
4:	learn: 0.6123348	test: 0.6114598	best: 0.6114598 (4)	total: 402ms	remaining: 1m 19s
5:	learn: 0.6009395	test: 0.6000390	best: 0.6000390 (5)	total: 474ms	remaining: 1m 18s
6:	learn: 0.5907108	test: 0.5898023	best: 0.5898023 (6)	total: 565ms	remaining: 1m 20s
7:	learn: 0.5814500	test: 0.5807468	best: 0.5807468 (7)	total: 644ms	remaining: 1m 19s
8:	learn: 0.5732775	test: 0.5726102	best: 0.5726102 (8)	total: 722ms	remaining: 1m 19s
9:	learn: 0.5659380	test: 0.5654216	best: 0.5654216 (9)	total: 804ms	remaining: 1m 19s
10:	learn: 0.5593899	test: 0.5590957	best: 0.5590957 (10)	total: 885ms	remaining: 1m 19s
11:	learn: 0.5533981	test: 0.5533275	bes

93:	learn: 0.4930249	test: 0.4965684	best: 0.4965684 (93)	total: 7.96s	remaining: 1m 16s
94:	learn: 0.4929590	test: 0.4965347	best: 0.4965347 (94)	total: 8.04s	remaining: 1m 16s
95:	learn: 0.4928943	test: 0.4964781	best: 0.4964781 (95)	total: 8.15s	remaining: 1m 16s
96:	learn: 0.4928288	test: 0.4964284	best: 0.4964284 (96)	total: 8.22s	remaining: 1m 16s
97:	learn: 0.4927687	test: 0.4963729	best: 0.4963729 (97)	total: 8.31s	remaining: 1m 16s
98:	learn: 0.4927001	test: 0.4963270	best: 0.4963270 (98)	total: 8.39s	remaining: 1m 16s
99:	learn: 0.4926307	test: 0.4962569	best: 0.4962569 (99)	total: 8.48s	remaining: 1m 16s
100:	learn: 0.4925733	test: 0.4962102	best: 0.4962102 (100)	total: 8.57s	remaining: 1m 16s
101:	learn: 0.4925183	test: 0.4961572	best: 0.4961572 (101)	total: 8.65s	remaining: 1m 16s
102:	learn: 0.4924575	test: 0.4961091	best: 0.4961091 (102)	total: 8.74s	remaining: 1m 16s
103:	learn: 0.4924091	test: 0.4960693	best: 0.4960693 (103)	total: 8.83s	remaining: 1m 16s
104:	learn: 0

184:	learn: 0.4891375	test: 0.4939820	best: 0.4939820 (184)	total: 15.8s	remaining: 1m 9s
185:	learn: 0.4890966	test: 0.4939453	best: 0.4939453 (185)	total: 15.9s	remaining: 1m 9s
186:	learn: 0.4890639	test: 0.4939359	best: 0.4939359 (186)	total: 16s	remaining: 1m 9s
187:	learn: 0.4890434	test: 0.4939233	best: 0.4939233 (187)	total: 16.1s	remaining: 1m 9s
188:	learn: 0.4890007	test: 0.4939069	best: 0.4939069 (188)	total: 16.1s	remaining: 1m 9s
189:	learn: 0.4889799	test: 0.4939047	best: 0.4939047 (189)	total: 16.2s	remaining: 1m 9s
190:	learn: 0.4889419	test: 0.4938793	best: 0.4938793 (190)	total: 16.3s	remaining: 1m 9s
191:	learn: 0.4889071	test: 0.4938636	best: 0.4938636 (191)	total: 16.4s	remaining: 1m 9s
192:	learn: 0.4888630	test: 0.4938405	best: 0.4938405 (192)	total: 16.5s	remaining: 1m 9s
193:	learn: 0.4888215	test: 0.4938103	best: 0.4938103 (193)	total: 16.6s	remaining: 1m 8s
194:	learn: 0.4887978	test: 0.4937963	best: 0.4937963 (194)	total: 16.7s	remaining: 1m 8s
195:	learn: 

276:	learn: 0.4865967	test: 0.4929940	best: 0.4929940 (276)	total: 23.9s	remaining: 1m 2s
277:	learn: 0.4865787	test: 0.4929911	best: 0.4929911 (277)	total: 24s	remaining: 1m 2s
278:	learn: 0.4865629	test: 0.4929827	best: 0.4929827 (278)	total: 24.1s	remaining: 1m 2s
279:	learn: 0.4865362	test: 0.4929708	best: 0.4929708 (279)	total: 24.2s	remaining: 1m 2s
280:	learn: 0.4865018	test: 0.4929627	best: 0.4929627 (280)	total: 24.2s	remaining: 1m 1s
281:	learn: 0.4864799	test: 0.4929558	best: 0.4929558 (281)	total: 24.3s	remaining: 1m 1s
282:	learn: 0.4864577	test: 0.4929551	best: 0.4929551 (282)	total: 24.4s	remaining: 1m 1s
283:	learn: 0.4864408	test: 0.4929520	best: 0.4929520 (283)	total: 24.5s	remaining: 1m 1s
284:	learn: 0.4864148	test: 0.4929358	best: 0.4929358 (284)	total: 24.5s	remaining: 1m 1s
285:	learn: 0.4864001	test: 0.4929309	best: 0.4929309 (285)	total: 24.6s	remaining: 1m 1s
286:	learn: 0.4863627	test: 0.4929268	best: 0.4929268 (286)	total: 24.7s	remaining: 1m 1s
287:	learn: 

368:	learn: 0.4846565	test: 0.4924360	best: 0.4924301 (365)	total: 31.6s	remaining: 54s
369:	learn: 0.4846409	test: 0.4924341	best: 0.4924301 (365)	total: 31.7s	remaining: 53.9s
370:	learn: 0.4846165	test: 0.4924321	best: 0.4924301 (365)	total: 31.7s	remaining: 53.8s
371:	learn: 0.4845882	test: 0.4924226	best: 0.4924226 (371)	total: 31.8s	remaining: 53.7s
372:	learn: 0.4845659	test: 0.4924173	best: 0.4924173 (372)	total: 31.9s	remaining: 53.6s
373:	learn: 0.4845542	test: 0.4924040	best: 0.4924040 (373)	total: 32s	remaining: 53.5s
374:	learn: 0.4845373	test: 0.4923958	best: 0.4923958 (374)	total: 32.1s	remaining: 53.5s
375:	learn: 0.4845130	test: 0.4924003	best: 0.4923958 (374)	total: 32.2s	remaining: 53.4s
376:	learn: 0.4844844	test: 0.4924038	best: 0.4923958 (374)	total: 32.2s	remaining: 53.3s
377:	learn: 0.4844686	test: 0.4924026	best: 0.4923958 (374)	total: 32.3s	remaining: 53.2s
378:	learn: 0.4844405	test: 0.4924002	best: 0.4923958 (374)	total: 32.4s	remaining: 53.1s
379:	learn: 0.

460:	learn: 0.4828983	test: 0.4921690	best: 0.4921614 (459)	total: 39s	remaining: 45.6s
461:	learn: 0.4828845	test: 0.4921711	best: 0.4921614 (459)	total: 39.1s	remaining: 45.5s
462:	learn: 0.4828677	test: 0.4921722	best: 0.4921614 (459)	total: 39.2s	remaining: 45.4s
463:	learn: 0.4828542	test: 0.4921700	best: 0.4921614 (459)	total: 39.3s	remaining: 45.4s
464:	learn: 0.4828242	test: 0.4921608	best: 0.4921608 (464)	total: 39.4s	remaining: 45.3s
465:	learn: 0.4828097	test: 0.4921621	best: 0.4921608 (464)	total: 39.4s	remaining: 45.2s
466:	learn: 0.4827805	test: 0.4921535	best: 0.4921535 (466)	total: 39.5s	remaining: 45.1s
467:	learn: 0.4827603	test: 0.4921489	best: 0.4921489 (467)	total: 39.6s	remaining: 45s
468:	learn: 0.4827361	test: 0.4921466	best: 0.4921466 (468)	total: 39.7s	remaining: 45s
469:	learn: 0.4827184	test: 0.4921500	best: 0.4921466 (468)	total: 39.8s	remaining: 44.9s
470:	learn: 0.4827067	test: 0.4921440	best: 0.4921440 (470)	total: 39.9s	remaining: 44.8s
471:	learn: 0.48

552:	learn: 0.4812727	test: 0.4920315	best: 0.4920300 (551)	total: 46.5s	remaining: 37.6s
553:	learn: 0.4812575	test: 0.4920338	best: 0.4920300 (551)	total: 46.6s	remaining: 37.5s
554:	learn: 0.4812433	test: 0.4920330	best: 0.4920300 (551)	total: 46.7s	remaining: 37.4s
555:	learn: 0.4812229	test: 0.4920326	best: 0.4920300 (551)	total: 46.7s	remaining: 37.3s
556:	learn: 0.4812105	test: 0.4920222	best: 0.4920222 (556)	total: 46.8s	remaining: 37.2s
557:	learn: 0.4811896	test: 0.4920209	best: 0.4920209 (557)	total: 46.9s	remaining: 37.1s
558:	learn: 0.4811731	test: 0.4920286	best: 0.4920209 (557)	total: 47s	remaining: 37s
559:	learn: 0.4811532	test: 0.4920228	best: 0.4920209 (557)	total: 47s	remaining: 37s
560:	learn: 0.4811345	test: 0.4920135	best: 0.4920135 (560)	total: 47.1s	remaining: 36.9s
561:	learn: 0.4811211	test: 0.4920076	best: 0.4920076 (561)	total: 47.2s	remaining: 36.8s
562:	learn: 0.4811089	test: 0.4920084	best: 0.4920076 (561)	total: 47.3s	remaining: 36.7s
563:	learn: 0.4810

644:	learn: 0.4797035	test: 0.4918186	best: 0.4917950 (632)	total: 54.2s	remaining: 29.8s
645:	learn: 0.4796865	test: 0.4918163	best: 0.4917950 (632)	total: 54.3s	remaining: 29.8s
646:	learn: 0.4796661	test: 0.4918152	best: 0.4917950 (632)	total: 54.4s	remaining: 29.7s
647:	learn: 0.4796426	test: 0.4918175	best: 0.4917950 (632)	total: 54.5s	remaining: 29.6s
648:	learn: 0.4796230	test: 0.4918203	best: 0.4917950 (632)	total: 54.5s	remaining: 29.5s
649:	learn: 0.4796119	test: 0.4918167	best: 0.4917950 (632)	total: 54.6s	remaining: 29.4s
650:	learn: 0.4795906	test: 0.4918226	best: 0.4917950 (632)	total: 54.7s	remaining: 29.3s
651:	learn: 0.4795754	test: 0.4918223	best: 0.4917950 (632)	total: 54.8s	remaining: 29.2s
652:	learn: 0.4795603	test: 0.4918214	best: 0.4917950 (632)	total: 54.9s	remaining: 29.2s
653:	learn: 0.4795434	test: 0.4918220	best: 0.4917950 (632)	total: 54.9s	remaining: 29.1s
654:	learn: 0.4795277	test: 0.4918237	best: 0.4917950 (632)	total: 55s	remaining: 29s
655:	learn: 0.

In [47]:
from catboost.utils import get_roc_curve
import sklearn
from sklearn import metrics
from catboost import *

eval_pool = Pool(X_validation, y_validation, cat_features=cat_features)
curve = get_roc_curve(model, eval_pool)
(fpr, tpr, thresholds) = curve
roc_auc = sklearn.metrics.auc(fpr, tpr)
roc_auc

0.6826266842426262

In [48]:
importance = model.get_feature_importance()
imp = {}
for i, column in enumerate(X_train.columns):
    imp[column] = importance[i]
Counter(imp).most_common()  

[('ltv', 14.569995995169808),
 ('DisbursalDate', 8.680562914123716),
 ('Employee_code_ID', 7.733431614194617),
 ('PERFORM_CNS.SCORE.DESCRIPTION', 7.201910057504537),
 ('branch_id', 6.218838522786485),
 ('supplier_id', 5.263424512607715),
 ('cust_log_disbursed_amount', 5.02534673429679),
 ('Current_pincode_ID', 4.7712667205247845),
 ('Date.of.Birth', 4.359225754155916),
 ('Employment.Type', 3.741500013966707),
 ('PERFORM_CNS.SCORE', 3.3825628816905655),
 ('manufacturer_id', 3.210895035351554),
 ('CREDIT.HISTORY.LENGTH', 3.205600637999087),
 ('PRIMARY.INSTAL.AMT', 3.186976562278276),
 ('cust_disbursal_ratio', 2.8456345195628328),
 ('State_ID', 2.7386947254613654),
 ('NO.OF_INQUIRIES', 2.481052527492166),
 ('cust_log_asset_cost', 1.7706582276706586),
 ('PRI.DISBURSED.AMOUNT', 1.765178713077769),
 ('PRI.CURRENT.BALANCE', 1.7579569236738355),
 ('PRI.OVERDUE.ACCTS', 1.7436626362094843),
 ('DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 1.516014911046644),
 ('PRI.ACTIVE.ACCTS', 1.495250200777761),
 ('

In [ ]:
# check = train_df[["PERFORM_CNS.SCORE.DESCRIPTION", "loan_default"]]

In [ ]:
# check["defaulters"] = check.groupby("PERFORM_CNS.SCORE.DESCRIPTION")["loan_default"].transform("sum")
# check["totals"] = check.groupby("PERFORM_CNS.SCORE.DESCRIPTION")["loan_default"].transform("count")
# check["default perct"] = check["defaulters"] / check["totals"]

In [ ]:
# dd = list(train_df['Employment.Type'].unique())
# emp = dict(zip(dd, [i for i in range(len(dd))]))
# X_train['Employment.Type'] = X_train['Employment.Type'].apply(lambda x: emp[x])
# X_validation['Employment.Type'] = X_validation['Employment.Type'].apply(lambda x: emp[x])

In [ ]:
# test['Employment.Type'] = test['Employment.Type'].apply(lambda x: emp[x])

# Create submission

In [49]:
submission = pd.DataFrame()
submission['UniqueID'] = test_df.UniqueID
submission['loan_default'] = list(model.predict_proba(test_df.drop(['UniqueID'], axis=1))[:,1])
submission.to_csv('sam_sub3.csv', index=False)

## Debug